d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# MLflow Lab

In this lab we will explore the path to moving models to production with MLflow using the following steps:

1. Load in Airbnb dataset, and save both training dataset and test dataset as Delta tables
2. Train an MLlib linear regression model using all the listing features and tracking parameters, metrics artifacts and Delta table version to MLflow
3. Register this initial model and move it to staging using MLflow Model Registry
4. Add a new column, `log_price` to both our train and test table and update the corresponding Delta tables
5. Train a second MLlib linear regression model, this time using `log_price` as our target and training on all features, tracking to MLflow 
6. Compare the performance of the different runs by looking at the underlying data versions for both models
7. Move the better performing model to production in MLflow model registry

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) In this lab you:<br>
- Create Delta tables
- Track your MLlib model and Delta table version using MLflow
- Use MLflow model registry to version your models

In [0]:
# req for community edition
# will restart python interpreter
pip install mlflow

Python interpreter will be restarted.
Requirement already satisfied: mlflow in /local_disk0/.ephemeral_nfs/envs/pythonEnv-fc9c189a-af0f-4968-ad6f-a09ef01af4f0/lib/python3.8/site-packages (1.18.0)
Requirement already satisfied: pandas in /databricks/python3/lib/python3.8/site-packages (from mlflow) (1.1.5)
Requirement already satisfied: gunicorn; platform_system != "Windows" in /local_disk0/.ephemeral_nfs/envs/pythonEnv-fc9c189a-af0f-4968-ad6f-a09ef01af4f0/lib/python3.8/site-packages (from mlflow) (20.1.0)
Requirement already satisfied: gitpython>=2.1.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-fc9c189a-af0f-4968-ad6f-a09ef01af4f0/lib/python3.8/site-packages (from mlflow) (3.1.18)
Requirement already satisfied: numpy in /databricks/python3/lib/python3.8/site-packages (from mlflow) (1.19.2)
Requirement already satisfied: prometheus-flask-exporter in /local_disk0/.ephemeral_nfs/envs/pythonEnv-fc9c189a-af0f-4968-ad6f-a09ef01af4f0/lib/python3.8/site-packages (from mlflow) (0.18.2)
Requirement already satisfied: databricks-cli>=0.8.7 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-fc9c189a-af0f-4968-ad6f-a09ef01af4f0/lib/python3.8/site-packages (from mlflow) (0.14.3)
Requirement already satisfied: Flask in /local_disk0/.ephemeral_nfs/envs/pythonEnv-fc9c189a-af0f-4968-ad6f-a09ef01af4f0/lib/python3.8/site-packages (from mlflow) (2.0.1)
Requirement already satisfied: protobuf>=3.7.0 in /databricks/python3/lib/python3.8/site-packages (from mlflow) (3.17.0)
Requirement already satisfied: cloudpickle in /local_disk0/.ephemeral_nfs/envs/pythonEnv-fc9c189a-af0f-4968-ad6f-a09ef01af4f0/lib/python3.8/site-packages (from mlflow) (1.6.0)
Requirement already satisfied: pyyaml>=5.1 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-fc9c189a-af0f-4968-ad6f-a09ef01af4f0/lib/python3.8/site-packages (from mlflow) (5.4.1)
Requirement already satisfied: click>=7.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-fc9c189a-af0f-4968-ad6f-a09ef01af4f0/lib/python3.8/site-packages (from mlflow) (8.0.1)
Requirement already satisfied: sqlparse>=0.3.1 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-fc9c189a-af0f-4968-ad6f-a09ef01af4f0/lib/python3.8/site-packages (from mlflow) (0.4.1)
Requirement already satisfied: docker>=4.0.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-fc9c189a-af0f-4968-ad6f-a09ef01af4f0/lib/python3.8/site-packages (from mlflow) (5.0.0)
Requirement already satisfied: alembic<=1.4.1 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-fc9c189a-af0f-4968-ad6f-a09ef01af4f0/lib/python3.8/site-packages (from mlflow) (1.4.1)
Requirement already satisfied: querystring-parser in /local_disk0/.ephemeral_nfs/envs/pythonEnv-fc9c189a-af0f-4968-ad6f-a09ef01af4f0/lib/python3.8/site-packages (from mlflow) (1.2.4)
Requirement already satisfied: requests>=2.17.3 in /databricks/python3/lib/python3.8/site-packages (from mlflow) (2.24.0)
Requirement already satisfied: entrypoints in /databricks/python3/lib/python3.8/site-packages (from mlflow) (0.3)
Requirement already satisfied: packaging in /local_disk0/.ephemeral_nfs/envs/pythonEnv-fc9c189a-af0f-4968-ad6f-a09ef01af4f0/lib/python3.8/site-packages (from mlflow) (21.0)
Requirement already satisfied: sqlalchemy in /local_disk0/.ephemeral_nfs/envs/pythonEnv-fc9c189a-af0f-4968-ad6f-a09ef01af4f0/lib/python3.8/site-packages (from mlflow) (1.4.20)
Requirement already satisfied: pytz in /databricks/python3/lib/python3.8/site-packages (from mlflow) (2020.5)
Requirement already satisfied: python-dateutil>=2.7.3 in /databricks/python3/lib/python3.8/site-packages (from pandas->mlflow) (2.8.1)
Requirement already satisfied: setuptools>=3.0 in /databricks/python3/lib/python3.8/site-packages (from gunicorn; platform_system != "Windows"->mlflow) (50.3.1)
Requirement already satisfied: gitdb<5,>=4.0.1 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-fc9c189a-af0f-4968-ad6f-a09ef01af4f0/lib/python3.8/site-packages (from gitpython>=2.1.0->mlflow) (4.0.7)
Requirement already satisfied: prometheus-client in /local_disk0/.ephemeral_nfs/envs/python

In [0]:
%run "../Includes/Classroom-Setup"

Initialized classroom variables & functions...

Datasets are already mounted to /mnt/training from s3a://databricks-corp-training/common

Declared various utility methods: 
 Declared untilStreamIsReady( name:String ) to control workflow 
 
 All done!

###  Step 1. Creating Delta Tables

Data versioning is an advantage of using Delta Lake, which preserves previous versions of datasets so that you can restore later.

Let's split our dataset into train and test datasets, and writing them out in Delta format. You can read more at the Delta Lake [documentation](https://docs.delta.io/latest/index.html).

In [0]:
filePath = "dbfs:/mnt/training/airbnb/sf-listings/sf-listings-2019-03-06-clean.delta/"
airbnbDF = spark.read.format("delta").load(filePath)

(trainDF, testDF) = airbnbDF.randomSplit([.8, .2], seed=42)

In [0]:
trainDeltaPath = userhome + "/machine-learning-p/train.delta"
testDeltaPath = userhome + "/machine-learning-p/test.delta"

# In case paths already exists
dbutils.fs.rm(trainDeltaPath, True)
dbutils.fs.rm(testDeltaPath, True)

(trainDF
  .write
  .mode("overwrite")
  .format("delta")
  .save(trainDeltaPath)
)

(testDF
  .write
  .mode("overwrite")
  .format("delta")
  .save(testDeltaPath)
)

Let's now read in our train and test Delta tables, specifying that we want the first version of these tables. This [blog post](https://databricks.com/blog/2019/02/04/introducing-delta-time-travel-for-large-scale-data-lakes.html) has a great example of how to read in a Delta table at a given version.

In [0]:
# TODO
data_version = 0
trainDelta = spark.read.format("delta").option("versionAsOf", data_version).load(trainDeltaPath)
testDelta = spark.read.format("delta").option("versionAsOf", data_version).load(testDeltaPath)

### Review Delta Table History
All the transactions for this table are stored within this table including the initial set of insertions, update, delete, merge, and inserts.

In [0]:
spark.sql(f"CREATE DATABASE IF NOT EXISTS {cleaned_username}")
spark.sql(f"USE {cleaned_username}")
spark.sql("DROP TABLE IF EXISTS train_delta")
spark.sql(f"CREATE TABLE train_delta USING DELTA LOCATION '{trainDeltaPath}'")

Out[9]: DataFrame[]

In [0]:
%sql
DESCRIBE HISTORY train_delta

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata
0,2021-07-05T12:34:01.000+0000,3311790168177,melburne42@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(1750467221661667),0705-121312-elite555,null,WriteSerializable,false,"Map(numFiles -> 4, numOutputBytes -> 205120, numOutputRows -> 5786)",null


-sandbox

By default Delta tables [keep a commit history of 30 days](https://docs.databricks.com/delta/delta-batch.html#data-retention). This retention period can be adjusted by setting `delta.logRetentionDuration`, which will determine how far back in time you can go. Note that setting this can result in storage costs to go up. 

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> Be aware that versioning with Delta in this manner may not be feasible as a long term solution. The retention period of Delta tables can be increased, but with that comes additional costs to storage. Alternative methods of data versioning when training models and tracking to MLflow is to save copies of the datasets, either as an MLflow artifact (for a small dataset), or save to a separate distributed location and record the location of the underlying dataset as a tag in MLflow

### Step 2. Log initial run to MLflow and move to staging in Model Registry

Let's first log a run to MLflow where we use all features. We use the same approach with RFormula as before. This time however, let's also log both the version of our data and the data path to MLflow.

In [0]:
# TODO
import mlflow
import mlflow.spark
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import RFormula

with mlflow.start_run(run_name="lr_model") as run:
  
  # Log parameters
  mlflow.log_param("data_path", trainDeltaPath)  
  # TODO: Log label: price-all-features
  mlflow.log_param("label", "price-all-features")
  # TODO: Log data_version: data_version
  mlflow.log_param("data_version", data_version)
  
    
  # Create pipeline
  rFormula = RFormula(formula="price ~ .", featuresCol="features", labelCol="price", handleInvalid="skip")
  lr = LinearRegression(labelCol="price", featuresCol="features")
  pipeline = Pipeline(stages = [rFormula, lr])
  model = pipeline.fit(trainDelta)
  
  # Log pipeline
  # TODO: Log model: model
  mlflow.spark.log_model(model, "model")

  # Create predictions and metrics
  predDF = model.transform(testDelta)
  regressionEvaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction")
  rmse = regressionEvaluator.setMetricName("rmse").evaluate(predDF)
  r2 = regressionEvaluator.setMetricName("r2").evaluate(predDF)
  
  # Log metrics
  # TODO: Log RMSE
  mlflow.log_metric("rmse", rmse)
  # TODO: Log R2
  mlflow.log_metric("r2", r2)
  
  runID = run.info.run_id

### Step 3. Register model and move to staging using MLflow Model Registry

-sandbox

We are happy with the performance of the above model and want to move it to staging. Let's create the model and register it to the MLflow model registry.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> Make sure the path to `model_uri` matches the subdirectory (the second argument to `mlflow.log_model()`) included above.

In [0]:
model_name = f"{cleaned_username}_mllib_lr"
model_uri = f"runs:/{runID}/model" # NOTE: confirm that this path is corrrect

model_details = mlflow.register_model(model_uri=model_uri, name=model_name)

--------------------------------------------------------------------------- 
 RestException Traceback (most recent call last)
 <command-1750467221661686> in <module> 
 2 model_uri = f"runs:/{runID}/model" # NOTE: confirm that this path is corrrect 
 3 
 ----> 4 model_details = mlflow . register_model ( model_uri = model_uri , name = model_name ) 

 /local_disk0/.ephemeral_nfs/envs/pythonEnv-fc9c189a-af0f-4968-ad6f-a09ef01af4f0/lib/python3.8/site-packages/mlflow/tracking/_model_registry/fluent.py in register_model (model_uri, name, await_registration_for) 
 65 )
 66 else : 
 ---> 67 raise e
 68 
 69 if RunsArtifactRepository . is_runs_uri ( model_uri ) : 

 /local_disk0/.ephemeral_nfs/envs/pythonEnv-fc9c189a-af0f-4968-ad6f-a09ef01af4f0/lib/python3.8/site-packages/mlflow/tracking/_model_registry/fluent.py in register_model (model_uri, name, await_registration_for) 
 56 client = MlflowClient ( ) 
 57 try : 
 ---> 58 create_model_response = client . create_registered_model ( name ) 
 59 eprint ( "Successfully registered model '%s'." % create_model_response . name ) 
 60 except MlflowException as e : 

 /local_disk0/.ephemeral_nfs/envs/pythonEnv-fc9c189a-af0f-4968-ad6f-a09ef01af4f0/lib/python3.8/site-packages/mlflow/tracking/client.py in create_registered_model (self, name, tags, description) 
 1636 description : This sentiment analysis model classifies the tone - happy , sad , angry . 
 1637 """
 -> 1638 return self . _get_registry_client ( ) . create_registered_model ( name , tags , description ) 
 1639 
 1640 def rename_registered_model ( self , name : str , new_name : str ) -> RegisteredModel : 

 /local_disk0/.ephemeral_nfs/envs/pythonEnv-fc9c189a-af0f-4968-ad6f-a09ef01af4f0/lib/python3.8/site-packages/mlflow/tracking/_model_registry/client.py in create_registered_model (self, name, tags, description) 
 50 tags = tags if tags else { } 
 51 tags = [ RegisteredModelTag ( key , str ( value ) ) for key , value in tags . items ( ) ] 
 ---> 52 return self . store . create_registered_model ( name , tags , description ) 
 53 
 54 def update_registered_model ( self , name , description ) : 

 /local_disk0/.ephemeral_nfs/envs/pythonEnv-fc9c189a-af0f-4968-ad6f-a09ef01af4f0/lib/python3.8/site-packages/mlflow/store/model_registry/rest_store.py in create_registered_model (self, name, tags, description) 
 75 CreateRegisteredModel ( name = name , tags = proto_tags , description = description ) 
 76 )
 ---> 77 response_proto = self . _call_endpoint ( CreateRegisteredModel , req_body ) 
 78 return RegisteredModel . from_proto ( response_proto . registered_model ) 
 79 

 /local_disk0/.ephemeral_nfs/envs/pythonEnv-fc9c189a-af0f-4968-ad6f-a09ef01af4f0/lib/python3.8/site-packages/mlflow/store/model_registry/rest_store.py in _call_endpoint (self, api, json_body) 
 56 endpoint , method = _METHOD_TO_INFO [ api ] 
 57 response_proto = api . Response ( ) 
 ---> 58 return call_endpoint ( self . get_host_creds ( ) , endpoint , method , json_body , response_proto ) 
 59 
 60 # CRUD API for RegisteredModel objects 

 /local_disk0/.ephemeral_nfs/envs/pythonEnv-fc9c189a-af0f-4968-ad6f-a09ef01af4f0/lib/python3.8/site-packages/mlflow/utils/rest_utils.py in call_endpoint (host_creds, endpoint, method, json_body, response_proto) 
 168 host_creds = host_creds , endpoint = endpoint , method = method , json = json_body
 169 )
 --> 170 response = verify_rest_response ( response , endpoint ) 
 171 js_dict = json . loads ( response . text ) 
 172 parse_dict ( js_dict = js_dict , message = response_proto ) 

 /local_disk0/.ephemeral_nfs/envs/pythonEnv-fc9c189a-af0f-4968-ad6f-a09ef01af4f0/lib/python3.8/site-packages/mlflow/utils/rest_utils.py in verify_rest_response (response, endpoint) 
 120 if response . status_code != 200 : 
 121 if _can_parse_as_json ( response . text ) : 
 --> 122 raise RestException ( json . loads ( response . text ) ) 
 123 else : 
 124 base_msg = "API request to endpoint %s failed with error code " "%s != 200" % (

 RestException : PERMISSION_DE

Check the current status of the model.

In [0]:
from mlflow.tracking.client import MlflowClient

client = MlflowClient()
model_version_details = client.get_model_version(name=model_name, version=1)

model_version_details.status

--------------------------------------------------------------------------- 
 RestException Traceback (most recent call last)
 <command-1750467221661688> in <module> 
 2 
 3 client = MlflowClient ( ) 
 ----> 4 model_version_details = client . get_model_version ( name = model_name , version = 1 ) 
 5 
 6 model_version_details . status

 /local_disk0/.ephemeral_nfs/envs/pythonEnv-fc9c189a-af0f-4968-ad6f-a09ef01af4f0/lib/python3.8/site-packages/mlflow/tracking/client.py in get_model_version (self, name, version) 
 2523 Version : 2 
 2524 """
 -> 2525 return self . _get_registry_client ( ) . get_model_version ( name , version ) 
 2526 
 2527 def get_model_version_download_uri ( self , name : str , version : str ) -> str : 

 /local_disk0/.ephemeral_nfs/envs/pythonEnv-fc9c189a-af0f-4968-ad6f-a09ef01af4f0/lib/python3.8/site-packages/mlflow/tracking/_model_registry/client.py in get_model_version (self, name, version) 
 262 : return : A single : py : class : ` mlflow . entities . model_registry . ModelVersion ` object . 
 263 """
 --> 264 return self . store . get_model_version ( name , version ) 
 265 
 266 def delete_model_version ( self , name , version ) : 

 /local_disk0/.ephemeral_nfs/envs/pythonEnv-fc9c189a-af0f-4968-ad6f-a09ef01af4f0/lib/python3.8/site-packages/mlflow/store/model_registry/rest_store.py in get_model_version (self, name, version) 
 308 """
 309 req_body = message_to_json ( GetModelVersion ( name = name , version = str ( version ) ) ) 
 --> 310 response_proto = self . _call_endpoint ( GetModelVersion , req_body ) 
 311 return ModelVersion . from_proto ( response_proto . model_version ) 
 312 

 /local_disk0/.ephemeral_nfs/envs/pythonEnv-fc9c189a-af0f-4968-ad6f-a09ef01af4f0/lib/python3.8/site-packages/mlflow/store/model_registry/rest_store.py in _call_endpoint (self, api, json_body) 
 56 endpoint , method = _METHOD_TO_INFO [ api ] 
 57 response_proto = api . Response ( ) 
 ---> 58 return call_endpoint ( self . get_host_creds ( ) , endpoint , method , json_body , response_proto ) 
 59 
 60 # CRUD API for RegisteredModel objects 

 /local_disk0/.ephemeral_nfs/envs/pythonEnv-fc9c189a-af0f-4968-ad6f-a09ef01af4f0/lib/python3.8/site-packages/mlflow/utils/rest_utils.py in call_endpoint (host_creds, endpoint, method, json_body, response_proto) 
 168 host_creds = host_creds , endpoint = endpoint , method = method , json = json_body
 169 )
 --> 170 response = verify_rest_response ( response , endpoint ) 
 171 js_dict = json . loads ( response . text ) 
 172 parse_dict ( js_dict = js_dict , message = response_proto ) 

 /local_disk0/.ephemeral_nfs/envs/pythonEnv-fc9c189a-af0f-4968-ad6f-a09ef01af4f0/lib/python3.8/site-packages/mlflow/utils/rest_utils.py in verify_rest_response (response, endpoint) 
 120 if response . status_code != 200 : 
 121 if _can_parse_as_json ( response . text ) : 
 --> 122 raise RestException ( json . loads ( response . text ) ) 
 123 else : 
 124 base_msg = "API request to endpoint %s failed with error code " "%s != 200" % (

 RestException : PERMISSION_DENIED: Model Registry is not enabled for organization 7840814561969526.

Add a model description using [update_registered_model](https://mlflow.org/docs/latest/python_api/mlflow.tracking.html#mlflow.tracking.MlflowClient.update_registered_model).

In [0]:
# TODO
client.update_registered_model(
  name = model_details.name,
  description = "This model forecasts Airbnb housing prices"
)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1750467221661690> in <module> 
 1 # TODO 
 2 client.update_registered_model(
 ----> 3 name = model_details . name , 
 4 description = "This model forecasts Airbnb housing prices" 
 5 )

 NameError : name 'model_details' is not defined

###  Step 4. Feature Engineering: Evolve Data Schema

We now want to do some feature engineering with the aim of improving model performance; we can use Delta Lake to track older versions of the dataset. 

We will add `log_price` as a new column and update our Delta table with it.

In [0]:
from pyspark.sql.functions import col, log, exp

# Create a new log_price column for both train and test datasets
trainNew = trainDelta.withColumn("log_price", log(col("price")))
testNew = testDelta.withColumn("log_price", log(col("price")))

Save the updated DataFrames to `trainDeltaPath` and `testDeltaPath`, respectively, passing the `mergeSchema` option to safely evolve its schema.

In [0]:
# TODO
trainNew.write.option("mergeSchema", "true").format("delta").mode("overwrite").save(trainDeltaPath)
testNew.write.option("mergeSchema", "true").format("delta").mode("overwrite").save(testDeltaPath)

Look at the difference between the original & modified schemas

In [0]:
set(trainNew.schema.fields) ^ set(trainDelta.schema.fields)

Out[16]: {StructField(log_price,DoubleType,true)}

Let's review the Delta history of our `train_delta` table and load in the most recent versions of our train and test Delta tables.

In [0]:
%sql
DESCRIBE HISTORY train_delta

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata
1,2021-07-05T12:55:24.000+0000,3311790168177,melburne42@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(1750467221661667),0705-121312-elite555,0,WriteSerializable,false,"Map(numFiles -> 4, numOutputBytes -> 221980, numOutputRows -> 5786)",null
0,2021-07-05T12:34:01.000+0000,3311790168177,melburne42@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(1750467221661667),0705-121312-elite555,null,WriteSerializable,false,"Map(numFiles -> 4, numOutputBytes -> 205120, numOutputRows -> 5786)",null


In [0]:
data_version = 1
trainDeltaNew = spark.read.format("delta").option("versionAsOf", data_version).load(trainDeltaPath)  
testDeltaNew = spark.read.format("delta").option("versionAsOf", data_version).load(testDeltaPath)

### Step 5. Use `log_price` as target and track run with MLflow

Retrain the model on the updated data and compare its performance to the original, logging results to MLflow.

In [0]:
with mlflow.start_run(run_name="lr_log_model") as run:
  
  # Log parameters
  mlflow.log_param("label", "log-price")
  mlflow.log_param("data_version", data_version)
  mlflow.log_param("data_path", trainDeltaPath)    
 
  # Create pipeline
  rFormula = RFormula(formula="log_price ~ . - price", featuresCol="features", labelCol="log_price", handleInvalid="skip")  
  lr = LinearRegression(labelCol="log_price", predictionCol="log_prediction")
  pipeline = Pipeline(stages = [rFormula, lr])
  pipelineModel = pipeline.fit(trainDeltaNew)
   
  # Log model and update the registered model
  mlflow.spark.log_model(
    spark_model=pipelineModel,
    artifact_path="log-model",
    registered_model_name=model_name,
  )  
  
  # Create predictions and metrics
  predDF = pipelineModel.transform(testDelta)
  expDF = predDF.withColumn("prediction", exp(col("log_prediction")))
  rmse = regressionEvaluator.setMetricName("rmse").evaluate(expDF)
  r2 = regressionEvaluator.setMetricName("r2").evaluate(expDF)
  
  # Log metrics
  mlflow.log_metric("rmse", rmse)
  mlflow.log_metric("r2", r2)  
  
  runID = run.info.run_id

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /local_disk0/.ephemeral_nfs/envs/pythonEnv-fc9c189a-af0f-4968-ad6f-a09ef01af4f0/lib/python3.8/site-packages/mlflow/spark.py in log_model (spark_model, artifact_path, conda_env, dfs_tmpdir, sample_input, registered_model_name, signature, input_example, await_registration_for) 
 209 try : 
 --> 210 spark_model . save ( posixpath . join ( model_dir , _SPARK_MODEL_PATH_SUB ) ) 
 211 except Py4JJavaError : 

 /databricks/spark/python/pyspark/ml/util.py in save (self, path) 
 309 """Save this ML instance to the given path, a shortcut of 'write().save(path)'.""" 
 --> 310 self . write ( ) . save ( path ) 
 311 

 /databricks/spark/python/pyspark/ml/util.py in save (self, path) 
 260 raise TypeError ( "path should be a string, got type %s" % type ( path ) ) 
 --> 261 self . _jwrite . save ( path ) 
 262 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 116 try : 
 --> 117 return f ( * a , ** kw ) 
 118 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 325 if answer [ 1 ] == REFERENCE_TYPE : 
 --> 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 

 Py4JJavaError : An error occurred while calling o2458.save.
: java.rmi.RemoteException: java.lang.UnsupportedOperationException: No operations allowed on this path; nested exception is: 
	java.lang.UnsupportedOperationException: No operations allowed on this path
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:128)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:68)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV1.getFileStatus(DatabricksFileSystemV1.scala:270)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystem.getFileStatus(DatabricksFileSystem.scala:201)
	at org.apache.hadoop.fs.FileSystem.exists(FileSystem.java:1426)
	at org.apache.spark.ml.util.FileSystemOverwrite.handleOverwrite(ReadWrite.scala:675)
	at org.apache.spark.ml.util.MLWriter.save(ReadWrite.scala:167)
	at org.apache.spark.ml.PipelineModel$PipelineModelWriter.super$save(Pipeline.scala:344)
	at org.apache.spark.ml.PipelineModel$PipelineModelWriter.$anonfun$save$4(Pipeline.scala:344)
	at org.apache.spark.ml.MLEvents.withSaveInstanceEvent(events.scala:175)
	at org.apache.spark.ml.MLEvents.withSaveInstanceEvent$(events.scala:170)
	at org.apache.spark.ml.util.Instrumentation.withSaveInstanceEvent(Instrumentation.scala:43)
	at org.apache.spark.ml.PipelineModel$PipelineModelWriter.$anonfun$save$3(Pipeline.scala:344)
	at org.apache.spark.ml.PipelineModel$PipelineModelWriter.$anonfun$save$3$adapted(Pipeline.scala:344)
	at org.apache.spark.ml.util.Instrumentation$.$anonfun$instrumented$1(Instrumentation.scala:284)
	at scala.util.Try$.apply(Try.scala:213)
	at org.apache.spark.ml.util.Instrumentation$.instrumented(Instrumentation.scala:284)
	at org.apache.spark.ml.PipelineModel$PipelineModelWriter.save(Pipeline.scala:344)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands

### Step 6. Compare performance across runs by looking at Delta table versions

Use MLflow's [`mlflow.search_runs`](https://mlflow.org/docs/latest/python_api/mlflow.html#mlflow.search_runs) API to identify runs according to the version of data the run was trained on. Let's compare our runs according to our data versions.

Filter based on `params.data_path` and `params.data_version`.

In [0]:
# TODO
data_version = 0

mlflow.search_runs(filter_string=f"params.data_path='{trainDeltaPath}' and params.data_version='{data_version}'")

Out[19]:

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.rmse,metrics.r2,params.data_path,params.label,params.data_version,tags.mlflow.user,tags.mlflow.databricks.notebookRevisionID,tags.mlflow.source.name,tags.mlflow.databricks.notebookPath,tags.mlflow.runName,tags.mlflow.databricks.notebookID,tags.mlflow.source.type,tags.mlflow.log-model.history,tags.mlflow.databricks.cluster.info,tags.mlflow.databricks.cluster.id,tags.mlflow.databricks.webappURL,tags.mlflow.databricks.cluster.libraries
0,5aa205ad26054887a6d89aafcaaa8140,1750467221661667,FINISHED,dbfs:/databricks/mlflow-tracking/1750467221661...,2021-07-05 12:35:04.149000+00:00,2021-07-05 12:36:55.303000+00:00,133.462953,0.441615,dbfs:/user/melburne42@gmail.com/machine-learni...,price-all-features,0,melburne42@gmail.com,1625488615454,/Users/melburne42@gmail.com/Scalable-ML-3.4.1-...,/Users/melburne42@gmail.com/Scalable-ML-3.4.1-...,lr_model,1750467221661667,NOTEBOOK,"[{""artifact_path"":""model"",""flavors"":{""spark"":{...","{""cluster_name"":""My Cluster"",""spark_version"":""...",0705-121312-elite555,https://community.cloud.databricks.com,"{""installable"":[],""redacted"":[]}"


In [0]:
# TODO
data_version = 1

mlflow.search_runs(filter_string=f"params.data_path='{trainDeltaPath}' and params.data_version='{data_version}'")

Out[20]:

,run_id,experiment_id,status,artifact_uri,start_time,end_time,params.data_path,params.label,params.data_version,tags.mlflow.user,tags.mlflow.databricks.notebookRevisionID,tags.mlflow.source.name,tags.mlflow.databricks.notebookPath,tags.mlflow.runName,tags.mlflow.databricks.notebookID,tags.mlflow.source.type,tags.mlflow.log-model.history,tags.mlflow.databricks.cluster.info,tags.mlflow.databricks.cluster.id,tags.mlflow.databricks.webappURL,tags.mlflow.databricks.cluster.libraries
0,d60d693376ad441d87d0cbb1a1d7eb60,1750467221661667,FAILED,dbfs:/databricks/mlflow-tracking/1750467221661...,2021-07-05 12:57:34.325000+00:00,2021-07-05 12:59:04.796000+00:00,dbfs:/user/melburne42@gmail.com/machine-learni...,log-price,1,melburne42@gmail.com,1625489944931,/Users/melburne42@gmail.com/Scalable-ML-3.4.1-...,/Users/melburne42@gmail.com/Scalable-ML-3.4.1-...,lr_log_model,1750467221661667,NOTEBOOK,"[{""artifact_path"":""log-model"",""flavors"":{""spar...","{""cluster_name"":""My Cluster"",""spark_version"":""...",0705-121312-elite555,https://community.cloud.databricks.com,"{""installable"":[],""redacted"":[]}"


Which version of the data produced the best model?

### Step 7. Move best performing model to production using MLflow model registry

Get the most recent model version and move it to production

In [0]:
model_version_infos = client.search_model_versions(f"name = '{model_name}'")
new_model_version = max([model_version_info.version for model_version_info in model_version_infos])

--------------------------------------------------------------------------- 
 RestException Traceback (most recent call last)
 <command-1750467221661710> in <module> 
 ----> 1 model_version_infos = client . search_model_versions ( f"name = '{model_name}'" ) 
 2 new_model_version = max ( [ model_version_info . version for model_version_info in model_version_infos ] ) 

 /local_disk0/.ephemeral_nfs/envs/pythonEnv-fc9c189a-af0f-4968-ad6f-a09ef01af4f0/lib/python3.8/site-packages/mlflow/tracking/client.py in search_model_versions (self, filter_string) 
 2609 name = CordobaWeatherForecastModel ; run_id = e14afa2f47a040728060c1699968fd43 ; version = 2 
 2610 """
 -> 2611 return self . _get_registry_client ( ) . search_model_versions ( filter_string ) 
 2612 
 2613 def get_model_version_stages(

 /local_disk0/.ephemeral_nfs/envs/pythonEnv-fc9c189a-af0f-4968-ad6f-a09ef01af4f0/lib/python3.8/site-packages/mlflow/tracking/_model_registry/client.py in search_model_versions (self, filter_string) 
 292 : return : PagedList of : py : class : ` mlflow . entities . model_registry . ModelVersion ` objects . 
 293 """
 --> 294 return self . store . search_model_versions ( filter_string ) 
 295 
 296 def get_model_version_stages ( self , name , version ) : 

 /local_disk0/.ephemeral_nfs/envs/pythonEnv-fc9c189a-af0f-4968-ad6f-a09ef01af4f0/lib/python3.8/site-packages/mlflow/store/model_registry/rest_store.py in search_model_versions (self, filter_string) 
 336 """
 337 req_body = message_to_json ( SearchModelVersions ( filter = filter_string ) ) 
 --> 338 response_proto = self . _call_endpoint ( SearchModelVersions , req_body ) 
 339 model_versions = [ ModelVersion . from_proto ( mvd ) for mvd in response_proto . model_versions ] 
 340 return PagedList ( model_versions , response_proto . next_page_token ) 

 /local_disk0/.ephemeral_nfs/envs/pythonEnv-fc9c189a-af0f-4968-ad6f-a09ef01af4f0/lib/python3.8/site-packages/mlflow/store/model_registry/rest_store.py in _call_endpoint (self, api, json_body) 
 56 endpoint , method = _METHOD_TO_INFO [ api ] 
 57 response_proto = api . Response ( ) 
 ---> 58 return call_endpoint ( self . get_host_creds ( ) , endpoint , method , json_body , response_proto ) 
 59 
 60 # CRUD API for RegisteredModel objects 

 /local_disk0/.ephemeral_nfs/envs/pythonEnv-fc9c189a-af0f-4968-ad6f-a09ef01af4f0/lib/python3.8/site-packages/mlflow/utils/rest_utils.py in call_endpoint (host_creds, endpoint, method, json_body, response_proto) 
 168 host_creds = host_creds , endpoint = endpoint , method = method , json = json_body
 169 )
 --> 170 response = verify_rest_response ( response , endpoint ) 
 171 js_dict = json . loads ( response . text ) 
 172 parse_dict ( js_dict = js_dict , message = response_proto ) 

 /local_disk0/.ephemeral_nfs/envs/pythonEnv-fc9c189a-af0f-4968-ad6f-a09ef01af4f0/lib/python3.8/site-packages/mlflow/utils/rest_utils.py in verify_rest_response (response, endpoint) 
 120 if response . status_code != 200 : 
 121 if _can_parse_as_json ( response . text ) : 
 --> 122 raise RestException ( json . loads ( response . text ) ) 
 123 else : 
 124 base_msg = "API request to endpoint %s failed with error code " "%s != 200" % (

 RestException : PERMISSION_DENIED: Model Registry is not enabled for organization 7840814561969526.

In [0]:
client.update_model_version(
  name=model_name,
  version=new_model_version,
  description="This model version was built using a MLlib Linear Regression model with all features and log_price as predictor."
)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1750467221661711> in <module> 
 1 client.update_model_version(
 2 name = model_name , 
 ----> 3 version = new_model_version , 
 4 description = "This model version was built using a MLlib Linear Regression model with all features and log_price as predictor." 
 5 )

 NameError : name 'new_model_version' is not defined

In [0]:
time.sleep(60) # In case the registration is still pending before move to Staging

client.transition_model_version_stage(
  name=model_name,
  version=new_model_version,
  stage="Staging"
)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1750467221661712> in <module> 
 ----> 1 time . sleep ( 60 ) # In case the registration is still pending before move to Staging 
 2 
 3 client.transition_model_version_stage(
 4 name = model_name , 
 5 version = new_model_version , 

 NameError : name 'time' is not defined

In [0]:
# TODO
# Move Model into Production
client.transition_model_version_stage(
  name=model_name,
  version=new_model_version,
  stage="Production"
)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1750467221661713> in <module> 
 3 client.transition_model_version_stage(
 4 name = model_name , 
 ----> 5 version = new_model_version , 
 6 stage = "Production" 
 7 )

 NameError : name 'new_model_version' is not defined

Have a look at the MLflow model registry UI to check that your models have been successfully registered. You should see that version 1 of your model is now in staging, with version 2 in production.

To finish the lab, let's clean up by archiving both model versions and deleting the whole model from the registry

In [0]:
client.transition_model_version_stage(
  name=model_name,
  version=1,
  stage="Archived",
)

client.transition_model_version_stage(
  name=model_name,
  version=2,
  stage="Archived",
)

time.sleep(10)

client.delete_registered_model(model_name)


--------------------------------------------------------------------------- 
 RestException Traceback (most recent call last)
 <command-1750467221661716> in <module> 
 ----> 1 client.transition_model_version_stage(
 2 name = model_name , 
 3 version = 1 , 
 4 stage = "Archived" , 
 5 )

 /local_disk0/.ephemeral_nfs/envs/pythonEnv-fc9c189a-af0f-4968-ad6f-a09ef01af4f0/lib/python3.8/site-packages/mlflow/tracking/client.py in transition_model_version_stage (self, name, version, stage, archive_existing_versions) 
 2389 Stage : Staging
 2390 """
 -> 2391 return self._get_registry_client().transition_model_version_stage(
 2392 name , version , stage , archive_existing_versions
 2393 )

 /local_disk0/.ephemeral_nfs/envs/pythonEnv-fc9c189a-af0f-4968-ad6f-a09ef01af4f0/lib/python3.8/site-packages/mlflow/tracking/_model_registry/client.py in transition_model_version_stage (self, name, version, stage, archive_existing_versions) 
 249 if stage . strip ( ) == "" : 
 250 raise MlflowException ( "The stage must not be an empty string." ) 
 --> 251 return self.store.transition_model_version_stage(
 252 name = name , 
 253 version = version , 

 /local_disk0/.ephemeral_nfs/envs/pythonEnv-fc9c189a-af0f-4968-ad6f-a09ef01af4f0/lib/python3.8/site-packages/mlflow/store/model_registry/rest_store.py in transition_model_version_stage (self, name, version, stage, archive_existing_versions) 
 270 )
 271 )
 --> 272 response_proto = self . _call_endpoint ( TransitionModelVersionStage , req_body ) 
 273 return ModelVersion . from_proto ( response_proto . model_version ) 
 274 

 /local_disk0/.ephemeral_nfs/envs/pythonEnv-fc9c189a-af0f-4968-ad6f-a09ef01af4f0/lib/python3.8/site-packages/mlflow/store/model_registry/rest_store.py in _call_endpoint (self, api, json_body) 
 56 endpoint , method = _METHOD_TO_INFO [ api ] 
 57 response_proto = api . Response ( ) 
 ---> 58 return call_endpoint ( self . get_host_creds ( ) , endpoint , method , json_body , response_proto ) 
 59 
 60 # CRUD API for RegisteredModel objects 

 /local_disk0/.ephemeral_nfs/envs/pythonEnv-fc9c189a-af0f-4968-ad6f-a09ef01af4f0/lib/python3.8/site-packages/mlflow/utils/rest_utils.py in call_endpoint (host_creds, endpoint, method, json_body, response_proto) 
 168 host_creds = host_creds , endpoint = endpoint , method = method , json = json_body
 169 )
 --> 170 response = verify_rest_response ( response , endpoint ) 
 171 js_dict = json . loads ( response . text ) 
 172 parse_dict ( js_dict = js_dict , message = response_proto ) 

 /local_disk0/.ephemeral_nfs/envs/pythonEnv-fc9c189a-af0f-4968-ad6f-a09ef01af4f0/lib/python3.8/site-packages/mlflow/utils/rest_utils.py in verify_rest_response (response, endpoint) 
 120 if response . status_code != 200 : 
 121 if _can_parse_as_json ( response . text ) : 
 --> 122 raise RestException ( json . loads ( response . text ) ) 
 123 else : 
 124 base_msg = "API request to endpoint %s failed with error code " "%s != 200" % (

 RestException : PERMISSION_DENIED: Model Registry is not enabled for organization 7840814561969526.

-sandbox
&copy; 2020 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>